# Entropy Pooling example
This notebook replicates the results of the original EP method (Table 4 and Table 7) from the Sequential Entropy Pooling Heuristics article, available on https://ssrn.com/abstract=3936392.

In [1]:
import numpy as np
import pandas as pd
import fortitudo.tech as ft

In [2]:
# Load data and print prior P&L stats (Table 1 and Table 5)
R = ft.load_pnl()
instrument_names = R.columns
R = R.values
S, I = R.shape

p = np.ones((S, 1)) / S
means_prior = p.T @ R
vols_prior = np.sqrt(p.T @ (R - means_prior)**2)
skews_prior = p.T @ ((R - means_prior) / vols_prior)**3
kurts_prior = p.T @ ((R - means_prior) / vols_prior)**4
corr_prior = np.corrcoef(R.T)

data_prior = np.hstack((
    np.round(means_prior.T * 100, 1), np.round(vols_prior.T * 100, 1),
    np.round(skews_prior.T, 2), np.round(kurts_prior.T, 2)))
prior_df = pd.DataFrame(
    data=data_prior,
    index=instrument_names,
    columns=['Mean', 'Volatility', 'Skewness', 'Kurtosis'])
display(prior_df)

corr_prior_df = pd.DataFrame(
    data=np.intc(np.round(corr_prior * 100)),
    index=enumerate(instrument_names, start=1),
    columns=range(1, I + 1))
display(corr_prior_df)

,Mean,Volatility,Skewness,Kurtosis
Gov & MBS,-0.7,3.2,0.10,3.02
Corp IG,-0.4,3.4,0.11,3.11
Corp HY,1.9,6.1,0.17,2.97
EM Debt,2.7,7.5,0.22,3.06
DM Equity,6.4,14.9,0.40,3.15
EM Equity,8.0,26.9,0.77,4.10
Private Equity,13.7,27.8,0.72,3.76
Infrastructure,5.9,10.8,0.31,3.19
Real Estate,4.3,8.1,0.23,3.09
Hedge Funds,4.8,7.2,0.20,3.05


,1,2,3,4,5,6,7,8,9,10
"(1, Gov & MBS)",100,60,0,30,-20,-10,-30,-10,-20,-20
"(2, Corp IG)",60,100,50,60,10,20,10,10,10,30
"(3, Corp HY)",0,50,100,60,60,69,59,30,30,70
"(4, EM Debt)",30,60,60,100,40,59,30,20,20,40
"(5, DM Equity)",-20,10,60,40,100,69,79,40,40,80
"(6, EM Equity)",-10,20,69,59,69,100,69,30,39,79
"(7, Private Equity)",-30,10,59,30,79,69,100,39,49,79
"(8, Infrastructure)",-10,10,30,20,40,30,39,100,40,40
"(9, Real Estate)",-20,10,30,20,40,39,49,40,100,50
"(10, Hedge Funds)",-20,30,70,40,80,79,79,40,50,100


# Entropy Pooling views
We then specify the same views as the article: mean of Private Equity is 10%, volatility of EM Equity is less than or equal to 20%, skewness of DM Equity is less than or equal to −0.75, kurtosis of DM Equity is greater than or equal to 3.5, and correlation between Corp HY and EM Debt is 50%.

In [3]:
mean_rows = R[:, 2:7].T
vol_rows = (R[:, 2:6] - means_prior[:, 2:6]).T**2
skew_row = ((R[:, 4] - means_prior[:, 4]) / vols_prior[:, 4])**3
kurt_row = ((R[:, 4] - means_prior[:, 4]) / vols_prior[:, 4])**4
corr_row = (R[:, 2] - means_prior[:, 2]) * (R[:, 3] - means_prior[:, 3])

A = np.vstack((np.ones((1, S)), mean_rows, vol_rows[0:-1, :], corr_row[np.newaxis, :]))
b = np.vstack(([1], means_prior[:, 2:6].T, [0.1], vols_prior[:, 2:5].T**2,
               [0.5 * vols_prior[0, 2] * vols_prior[0, 3]]))
G = np.vstack((vol_rows[-1, :], skew_row, -kurt_row))
h = np.array([[0.2**2], [-0.75], [-3.5]])

# Posterior probabilities, relative entropy, and effective number of scenarios
Next, we calculate the posterior probabilities q, relative entropy (RE), and effective number of scenarios (ENS). Means, volatilities, skewness, kurtosis, and the correlation matrix are then
recalculated using the posterior probabilities. Finally, we print the posterior results (Table 4 and Table 7).

In [4]:
q = ft.entropy_pooling(p, A, b, G, h)
relative_entropy = q.T @ (np.log(q) - np.log(p))
effective_number_scenarios = np.exp(-relative_entropy)

In [5]:
means_post = q.T @ R
vols_post = np.sqrt(q.T @ (R - means_post)**2)
skews_post = q.T @ ((R - means_post) / vols_post)**3
kurts_post = q.T @ ((R - means_post) / vols_post)**4
cov_post = np.cov(R, rowvar=False, aweights=q[:, 0])
vols_inverse = np.diag(vols_post[0, :]**-1)
corr_post = vols_inverse @ cov_post @ vols_inverse

In [6]:
data_post = np.hstack((
    np.round(means_post.T * 100, 1), np.round(vols_post.T * 100, 1),
    np.round(skews_post.T, 2), np.round(kurts_post.T, 2)))
post_df = pd.DataFrame(
    data=data_post,
    index=instrument_names,
    columns=['Mean', 'Volatility', 'Skewness', 'Kurtosis'])
display(post_df)

print(f'ENS = {np.round(effective_number_scenarios[0, 0] * 100, 2)}%.')
print(f'RE = {np.round(relative_entropy[0, 0] * 100, 2)}%.')

corr_post_df = pd.DataFrame(
    data=np.intc(np.round(corr_post * 100)),
    index=enumerate(instrument_names, start=1),
    columns=range(1, I + 1))
display(corr_post_df)

,Mean,Volatility,Skewness,Kurtosis
Gov & MBS,-0.6,3.2,0.06,2.91
Corp IG,-0.5,3.4,0.14,3.12
Corp HY,1.9,6.1,-0.06,2.97
EM Debt,2.7,7.5,0.13,3.07
DM Equity,6.4,14.9,-0.75,3.50
EM Equity,8.0,20.0,-0.22,3.34
Private Equity,10.0,24.3,0.12,3.17
Infrastructure,5.7,10.6,0.28,3.16
Real Estate,3.7,8.0,0.13,3.02
Hedge Funds,4.6,7.0,-0.62,3.81


ENS = 70.92%.
RE = 34.36%.


,1,2,3,4,5,6,7,8,9,10
"(1, Gov & MBS)",100,60,-2,35,-23,-10,-34,-10,-20,-24
"(2, Corp IG)",60,100,51,63,9,20,7,9,11,29
"(3, Corp HY)",-2,51,100,50,57,64,55,27,27,67
"(4, EM Debt)",35,63,50,100,31,51,16,16,15,29
"(5, DM Equity)",-23,9,57,31,100,66,76,37,38,79
"(6, EM Equity)",-10,20,64,51,66,100,62,27,36,75
"(7, Private Equity)",-34,7,55,16,76,62,100,38,47,76
"(8, Infrastructure)",-10,9,27,16,37,27,38,100,39,38
"(9, Real Estate)",-20,11,27,15,38,36,47,39,100,49
"(10, Hedge Funds)",-24,29,67,29,79,75,76,38,49,100


# Comments
The results for the sequential heuristics in https://ssrn.com/abstract=3936392 are not replicated as they are a part of Fortitudo Technologies' proprietary software, which also contains an elegant interface for handling the different views instead of manually specifying them through A, b, G, and h. The interested reader can replicate the results of the sequential heuristics by using the P&L simulation that follows with this package and the Entropy Pooling technology.

# License

In [7]:
# fortitudo.tech - Novel Investment Technologies.
# Copyright (C) 2021-2022 Fortitudo Technologies.

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU General Public License for more details.

# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.